# 模型的测试 笔记
教程视频链接：https://www.bilibili.com/video/BV1hE411t7RN

这篇笔记对应视频合集中的
- 完整模型验证套路

## 1.加载图片
使用`PIL.Images`加载图片，并将图片转换为tensor数据类型：

In [29]:
from PIL import Image
import torchvision

img_path = '../images/plane.png'
img = Image.open(fp=img_path, mode='r')

img = img.convert('RGB')#RGBA为4通道，包含图片的透明度信息

to_tensor = torchvision.transforms.ToTensor()
resize = torchvision.transforms.Resize((32, 32))
transform = torchvision.transforms.Compose([resize, to_tensor])

img = transform(img).cuda()
img.shape

torch.Size([3, 32, 32])

## 2.加载神经网络

In [30]:
import torch
class Tudui(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2), #这里一定记得加逗号
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Flatten(),
            torch.nn.Linear(in_features=1024, out_features=10)
        )
    def forward(self, x):
        return self.model(x)


model = torch.load(r'..\models\tudui_classification_100.pth')
model

C:\Users\Tommy Ge\AppData\Local\Temp\ipykernel_1616\2139448074.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(r'..\models\tudui_classification_100.p

Tudui(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=10, bias=True)
  )
)

## 3.传入图片进行验证
cuda上训练得到的模型，其数据集也应该添加到cuda上

In [34]:
print(img.shape)
img = torch.reshape(img, (1, 3, 32, 32))

model.eval()

with torch.no_grad():
    output = model(img)

torch.argmax(output)

torch.Size([1, 3, 32, 32])


tensor(0, device='cuda:0')